# Data Centre Modelling
## Ryan Jenkinson

### Imports

In [ ]:
from dataclasses import dataclass

import numpy as np
import pandas as pd
import plotly.express as px
import pypsa
import xarray as xr

In [ ]:
np.random.seed(42)  # for reproducibility

### Constants

In [ ]:
@dataclass
class Costs:
    capital_cost: float
    marginal_cost: float

### Cost Assumptions

In [ ]:
## TODO: Sensitivity analysis for costs
from typing import Literal

cost_scenario: Literal["low", "medium", "high"] = "medium"

## Wind costs
# https://www.gov.uk/government/publications/electricity-generation-costs-2023
# Using 2025 costs
KW_TO_MW = 1000
pre_development_costs_gbp_per_mw = {
    "low": 60 * KW_TO_MW,
    "medium": 130 * KW_TO_MW,
    "high": 200 * KW_TO_MW,
}
construction_costs_gbp_per_mw = {
    "low": 1300 * KW_TO_MW,
    "medium": 1500 * KW_TO_MW,
    "high": 2000 * KW_TO_MW,
}
infrastructure_costs_gbp = {
    "low": 56_000_000,
    "medium": 64_300_000,
    "high": 74_100_000,
}  # TODO: How does this get added into the model? Do we annuitise it? But the other costs are annuitised per MW... :thinking:
wind_insurance = 3000  # £ per MW per year (insurance cost)

wind_capex_per_mw = (
    pre_development_costs_gbp_per_mw[cost_scenario]
    + construction_costs_gbp_per_mw[cost_scenario]
    + wind_insurance
)  # £ per MW
wind_lifetime_years = 30
wind_discount_rate = 0.07
annuitised_wind_cost = (wind_capex_per_mw * wind_discount_rate) / (
    1 - (1 + wind_discount_rate) ** -wind_lifetime_years
)
wind_fixed_opex = 43300  # £ per MW per year (fixed O&M)
wind_variable_operation_and_management_mwh = 1  # £ per MWh_elec (non-fuel variable O&M)

connection_and_use_of_system_charges = 44800  # £ per MW per year (grid connection costs) [NOTE: Not used in model as a microgrid]
WIND_COSTS = Costs(
    capital_cost=annuitised_wind_cost + wind_fixed_opex,  # £ per MW per year
    marginal_cost=0.01,  # £ per MWh (Variable O&M, assumed small)
)

## Gas costs
# Assume OCGT (Open Cycle Gas Turbine) for simplicity.
# Fast Start-Up: Aeroderivative gas turbines, which are derived from jet engine technology, can ramp up to full power in a matter of minutes. This rapid response is crucial to bridge the gap between a utility power outage and the exhaustion of the facility's Uninterruptible Power Supply (UPS) battery backup.
# High Reliability
# Smaller Footprint: Compared to other power generation technologies with similar output, OCGTs have a relatively compact footprint, which is an important consideration for data centers where space can be at a premium.
# Lower Emissions than Diesel
# CCGTs are an alternative for highly efficient for continuous, baseload power generation but they have a much longer start-up time and a significantly larger and more complex footprint so we use OCGT costs here
# https://www.gov.uk/government/publications/electricity-generation-costs-2023

# To decide the costings we use the following methodology:
# Operating Hours: 500 hr vs. 2000 hr --> 500hr is more akin to "backup" generator. 2000hr is equivalent to a standard 8 hour workday every weekday. So we use the 500hr variant
# OCGT 100MW 500 hr -> Logical starting point. Most data centres use 10 x 10MW backups so modelling as a 100MW is a good proxy
# OCGT 299MW 500 hr --> common to model even larger data centres as after 300MW there is much more complicated regulations
# We use the OCGT 100MW

gas_reference_model: Literal["100MW", "299MW"] = (
    "100MW"  # Reference model for gas turbine costs
)

gas_fuel_efficiency = {
    "100MW": 0.34,  # Efficiency of the 100MW OCGT (MWh_elec / MWh_fuel)
    "299MW": 0.35,  # Efficiency of the 299MW OCGT (MWh_elec / MWh_fuel)
}
gas_pre_development_costs_gbp_per_mw = {
    "100MW": {"low": 80 * KW_TO_MW, "medium": 90 * KW_TO_MW, "high": 110 * KW_TO_MW},
    "299MW": {"low": 30 * KW_TO_MW, "medium": 40 * KW_TO_MW, "high": 40 * KW_TO_MW},
}
gas_construction_costs_gbp_per_mw = {
    "100MW": {"low": 600 * KW_TO_MW, "medium": 700 * KW_TO_MW, "high": 900 * KW_TO_MW},
    "299MW": {"low": 300 * KW_TO_MW, "medium": 500 * KW_TO_MW, "high": 800 * KW_TO_MW},
}
gas_infrastructure_costs_gbp = {
    "100MW": {"low": 7_200_000, "medium": 14_400_000, "high": 28_700_000},
    "299MW": {"low": 7_800_000, "medium": 15_500_000, "high": 31_000_000},
}  # TODO: How to add this into the model? Do we annuitise it? But the other costs are annuitised per MW... :thinking:
gas_insurance = {
    "100MW": 2900,  # £ per MW per year (insurance cost for 100MW OCGT)
    "299MW": 1800,  # £ per MW per year (insurance cost for 299MW OCGT)
}
gas_connection_use_of_system_charges = 2700  # NOTE: Not used in model as a microgrid

gas_capex_per_mw = (
    gas_pre_development_costs_gbp_per_mw[gas_reference_model][cost_scenario]
    + gas_construction_costs_gbp_per_mw[gas_reference_model][cost_scenario]
    + gas_insurance[gas_reference_model]
)
gas_lifetime_years = 25
gas_discount_rate = 0.07
annuitised_gas_cost = (gas_capex_per_mw * gas_discount_rate) / (
    1 - (1 + gas_discount_rate) ** -gas_lifetime_years
)
gas_fixed_opex = {
    "100MW": 11300,  # £ per MW per year (fixed O&M for 100MW OCGT)
    "299MW": 7300,  # £ per MW per year (fixed O&M for 299MW OCGT)
}[gas_reference_model]  # Fixed O&M cost based on the reference model
gas_fuel_price_mwh = 60  # £ per MWh_fuel (assumed fuel price) # TODO: Validate
gas_variable_operation_and_management_mwh = {
    "100MW": 2,  # £ per MWh_elec (non-fuel variable O&M for 100MW OCGT)
    "299MW": 1,  # £ per MWh_elec (non-fuel variable O&M for 299MW OCGT)
}[gas_reference_model]
gas_marginal_cost_fuel = (
    gas_fuel_price_mwh / gas_fuel_efficiency[gas_reference_model]
)  # £ per MWh_elec
GAS_COSTS = Costs(
    capital_cost=annuitised_gas_cost + gas_fixed_opex,  # £ per MW per year
    marginal_cost=gas_marginal_cost_fuel
    + gas_variable_operation_and_management_mwh,  # £ per MWh_elec
)

## BESS (Battery Energy Storage System) costs
# https://www.utilitydive.com/news/tariffs-to-spike-power-generation-costs-reports/750133/
# https://www.numberanalytics.com/blog/energy-storage-economics
bess_energy_capex_per_mwh = {
    "low": 140000,  # £ per MWh
    "medium": 170000,  # £ per MWh
    "high": 200000,  # £ per MWh
}
bess_power_capex_per_mw = {
    "low": 450000,  # £ per MW
    "medium": 540000,  # £ per MW
    "high": 630000,  # £ per MW
}
bess_lifetime_years = 15
bess_discount_rate = 0.07

annuitised_bess_energy_cost = (
    bess_energy_capex_per_mwh[cost_scenario] * bess_discount_rate
) / (1 - (1 + bess_discount_rate) ** -bess_lifetime_years)
bess_capital_cost_energy = (
    annuitised_bess_energy_cost  # Cost for energy capacity (Store)
)

annuitised_bess_power_cost = (
    bess_power_capex_per_mw[cost_scenario] * bess_discount_rate
) / (1 - (1 + bess_discount_rate) ** -bess_lifetime_years)
bess_fixed_opex_per_mw_power = 20000  # Fixed OPEX for power conversion part
bess_capital_cost_power = (
    annuitised_bess_power_cost + bess_fixed_opex_per_mw_power
)  # Cost for power capacity (Links)

bess_round_trip_efficiency = 0.87
standing_loss_per_hour = 0.001
bess_marginal_cost_links = 0.001  # Marginal cost for charging/discharging

BESS_COSTS = Costs(
    capital_cost=bess_capital_cost_energy,  # £ per MWh per year
    marginal_cost=0,
)
BESS_LINK_COSTS = Costs(
    capital_cost=bess_capital_cost_power,  # £ per MW per year
    marginal_cost=bess_marginal_cost_links,  # £ per MWh_elec
)

In [ ]:
# TODO: Put everything relative to this or do a plot that plots this at difference sizes of data centre
DATA_CENTRE_LOAD_MW = 115  # MW, constant demand including PUE

### Wind Availability

In [ ]:
wind_data = xr.load_dataset(
    "../data/profile_39_offwind-dc.nc",
)

In [ ]:
wind_profile_df = wind_data["profile"].to_dataframe()
wind_profile_df.reset_index(inplace=True)

wind_availability_list = wind_profile_df[wind_profile_df["bus"] == "GB2 0"][
    "profile"
].values

In [ ]:
wind_profile_df

### Model

In [ ]:
def gen_model(data_centre_load_mw: float):
    # Define simulation period: a full year, hourly
    snapshots = pd.date_range(
        start="2019-01-01 00:00", end="2019-12-31 23:00", freq="h"
    )
    n = pypsa.Network(snapshots=snapshots)

    # --- DEFINE CARRIERS ---
    n.add("Carrier", "AC")  # For AC buses
    n.add("Carrier", "gas", co2_emissions=0.185)  # tCO2/MWh_th, typical for natural gas
    n.add("Carrier", "offshore_wind")
    n.add("Carrier", "battery")  # Carrier for battery components

    # --- DEFINE BUSES ---
    n.add("Bus", "electricity_bus", carrier="AC")
    n.add("Bus", "bess_connection_bus", carrier="AC")

    # --- DEFINE LOAD ---
    # Load (Data Centre): The data centre is modelled as a load with a constant power demand.
    constant_demand_profile = pd.Series(data_centre_load_mw, index=n.snapshots)
    n.add(
        "Load", "data_centre_load", bus="electricity_bus", p_set=constant_demand_profile
    )

    # --- DEFINE GENERATORS ---
    # Generator (Offshore Wind Farm): The offshore wind farm's capacity is extendable.

    wind_availability_pu = pd.Series(wind_availability_list, index=n.snapshots)
    wind_capacity_factor = 0.61  # Target capacity factor for the wind farm
    wind_availability_pu = wind_capacity_factor * (
        wind_availability_pu / wind_availability_pu.mean()
    )
    wind_availability_pu = wind_availability_pu.clip(0, 1)

    n.add(
        "Generator",
        "offshore_wind_farm",
        bus="electricity_bus",
        carrier="offshore_wind",
        p_nom_extendable=True,  # Allow capacity to be extended. To cover what we need to for the data centre load
        capital_cost=WIND_COSTS.capital_cost,
        marginal_cost=WIND_COSTS.marginal_cost,
        p_max_pu=wind_availability_pu,
    )

    # Generator (Gas Turbine Backup): The gas turbine provides backup, and its capacity is also extendable.
    n.add(
        "Generator",
        "gas_turbine",
        bus="electricity_bus",
        carrier="gas",
        p_nom_extendable=True,  # Allow capacity to be extended to cover the data centre load
        committable=False,  # Cannot be both committable and extendable for LOPF capacity optimization. TODO: Check if this is correct
        capital_cost=GAS_COSTS.capital_cost,
        marginal_cost=GAS_COSTS.marginal_cost,
    )

    # --- DEFINE STORAGE (BESS) ---
    # Store component representing the energy capacity of the BESS, and each of the links for charging and discharging.
    n.add(
        "Store",
        "bess_storage",
        bus="bess_connection_bus",
        carrier="battery",
        e_nom_extendable=True,
        e_cyclic=True,
        standing_loss=standing_loss_per_hour,
        capital_cost=BESS_COSTS.capital_cost,
    )
    n.add(
        "Link",
        "bess_charger",
        bus0="electricity_bus",
        bus1="bess_connection_bus",
        carrier="battery",
        p_nom_extendable=True,
        capital_cost=BESS_LINK_COSTS.capital_cost,
        efficiency=bess_round_trip_efficiency**0.5,
        marginal_cost=BESS_LINK_COSTS.marginal_cost,
    )
    n.add(
        "Link",
        "bess_discharger",
        bus0="bess_connection_bus",
        bus1="electricity_bus",
        carrier="battery",
        p_nom_extendable=True,
        capital_cost=0,  # Assuming power CAPEX is fully on charger link, or split if preferred
        efficiency=bess_round_trip_efficiency**0.5,
        marginal_cost=BESS_LINK_COSTS.marginal_cost,
    )

    # --- CONSISTENCY CHECK & SOLVE ---

    # Perform consistency check
    try:
        n.consistency_check()
        print("Network consistency check passed before solving.")
    except Exception as e:
        print(f"Consistency check error before solving: {e}")

    return n

In [ ]:
# Ensure you have a solver installed (e.g., cbc, glpk, gurobi)
solver_name = "cbc"  # or 'glpk', 'gurobi', etc.


def solve_model(n: pypsa.Network, solver_name: str, print_results: bool = True):
    """Solve the PyPSA network model using the specified solver.

    Parameters:
    - n: PyPSA Network object
    - solver_name: Name of the solver to use (e.g., 'cbc', 'glpk')
    - print_results: Whether to print the results after solving
    """
    try:
        # Solve the model using the optimize interface, common with linopy backend
        # The `solve_model` method handles the optimization.
        n.optimize(solver_name=solver_name)
        if not print_results:
            return n
        print("Model solved successfully.")

        # --- DISPLAY RESULTS ---
        print("\n--- Optimization Results ---")
        # Objective value is typically stored in n.model.objective after solving with n.optimize
        obj_value = n.model.objective.value
        print(f"Objective Value: {obj_value:.2f} £ (Total Cost)")

        print("\nOptimal Capacities:")
        # Check if components exist in results before trying to access them
        # Optimal capacities are in p_nom_opt (for Generators, Links) and e_nom_opt (for Stores)
        if (
            "offshore_wind_farm" in n.generators.index
            and "p_nom_opt" in n.generators.columns
        ):
            print(
                f"  Offshore Wind Farm (p_nom_opt): {n.generators.loc['offshore_wind_farm', 'p_nom_opt']:.2f} MW"
            )
        if "gas_turbine" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  Gas Turbine (p_nom_opt): {n.generators.loc['gas_turbine', 'p_nom_opt']:.2f} MW"
            )
        if "bess_storage" in n.stores.index and "e_nom_opt" in n.stores.columns:
            print(
                f"  BESS Storage Energy Capacity (e_nom_opt): {n.stores.loc['bess_storage', 'e_nom_opt']:.2f} MWh"
            )
        if "bess_charger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Charger Power Capacity (p_nom_opt): {n.links.loc['bess_charger', 'p_nom_opt']:.2f} MW"
            )
        if "bess_discharger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Discharger Power Capacity (p_nom_opt): {n.links.loc['bess_discharger', 'p_nom_opt']:.2f} MW"
            )

    except Exception as e:
        print(f"An error occurred during model solution or results processing: {e}")
        print("Please ensure you have a compatible solver installed and in your PATH.")
        print("Common solvers: cbc, glpk. You might need to install them.")
        print(
            "Also ensure Linopy (PyPSA's modeling backend) is installed and functioning correctly."
        )
    return n

In [ ]:
def get_optimal_capacities(n: pypsa.Network):
    """Get the optimal capacities of the network components after solving."""
    optimal_capacities = {
        "offshore_wind_farm": n.generators.loc["offshore_wind_farm", "p_nom_opt"],
        "gas_turbine": n.generators.loc["gas_turbine", "p_nom_opt"],
        "bess_storage": n.stores.loc["bess_storage", "e_nom_opt"],
        "bess_charger": n.links.loc["bess_charger", "p_nom_opt"],
        "bess_discharger": n.links.loc["bess_discharger", "p_nom_opt"],
    }
    return optimal_capacities


def get_asset_expenditure(n: pypsa.Network):
    stats = n.statistics()
    return {
        "gas": (
            stats.loc[("Generator", "gas"), "Capital Expenditure"]
            + stats.loc[("Generator", "gas"), "Operational Expenditure"]
        ),
        "offshore_wind": (
            stats.loc[("Generator", "offshore_wind"), "Capital Expenditure"]
            + stats.loc[("Generator", "offshore_wind"), "Operational Expenditure"]
        ),
        "battery": (
            stats.loc[("Link", "battery"), "Capital Expenditure"]
            + stats.loc[("Link", "battery"), "Operational Expenditure"]
            + stats.loc[("Store", "battery"), "Capital Expenditure"]
            + stats.loc[("Store", "battery"), "Operational Expenditure"]
        ),
    }

# Sensitivity analysis for different DC loads

In [ ]:
networks = {}
DC_LOADS = [10, 40, 100, 500]  # Different data centre load scenarios in MW

for i, dc_load_value in enumerate(DC_LOADS):
    n = gen_model(data_centre_load_mw=dc_load_value)
    n = solve_model(n, solver_name="highs", print_results=False)
    networks[dc_load_value] = n

In [ ]:
capacities = {load: get_optimal_capacities(n) for load, n in networks.items()}
capacities_df = pd.DataFrame(capacities).T  # Transpose so loads are rows
capacities_df.index.name = "data_centre_load_MW"
capacities_df.reset_index(inplace=True)
capacities_df

In [ ]:
fig = px.line(
    data_frame=capacities_df,
    x="data_centre_load_MW",
    y=capacities_df.columns,
    title="Generator Power Output per DC scenario",
    labels={"value": "Power (MW)", "variable": "Generator"},
    template="plotly_white",
)
fig.show()

In [ ]:
costs = {load: n.objective for load, n in networks.items()}

In [ ]:
costs_df = pd.DataFrame.from_dict(costs, orient="index")
costs_df.index.name = "data_centre_load_MW"
costs_df.reset_index(inplace=True)
costs_df.columns = [
    "data_centre_load_MW",
    "objective cost (GBP)",
]  # Rename columns for clarity
costs_df["cost_per_dc_mw"] = (
    costs_df["objective cost (GBP)"] / costs_df["data_centre_load_MW"]
)
costs_df

In [ ]:
asset_expenditures = {load: get_asset_expenditure(n) for load, n in networks.items()}
asset_expenditures_df = pd.DataFrame(
    asset_expenditures
).T  # Transpose so loads are rows
asset_expenditures_df.index.name = "data_centre_load_MW"
asset_expenditures_df.reset_index(inplace=True)
asset_expenditures_df

In [ ]:
fig = px.line(
    data_frame=asset_expenditures_df,
    x="data_centre_load_MW",
    y=asset_expenditures_df.columns,
    title="Total Asset Expenditures per DC scenario",
    labels={"value": "Cost (GBP)", "variable": "Generator"},
    template="plotly_white",
)
fig.show()

# Single datacentre exploration

In [ ]:
n = gen_model(data_centre_load_mw=DATA_CENTRE_LOAD_MW)
n = solve_model(n, solver_name="highs", print_results=True)

In [ ]:
generator_power = n.generators_t.p
data_centre_power = n.loads_t.p["data_centre_load"]
battery_power = n.links_t.p1["bess_charger"] - n.links_t.p1["bess_discharger"]

In [ ]:
asset_power = generator_power.copy()
asset_power["bess"] = battery_power
asset_power["dc_load"] = data_centre_power
asset_power

In [ ]:
fig = px.line(
    data_frame=asset_power,
    x=asset_power.index,
    y=asset_power.columns,
    title="Asset Power Output Over Time",
    labels={"value": "Power (MW)", "variable": "Asset"},
    template="plotly_white",
)
fig.show()

In [ ]:
fig = px.line(
    data_frame=pd.DataFrame(battery_power),
    x=data_centre_power.index,
    y=0,
    title="BESS Power Output Over Time",
    labels={"value": "Power (MW)", "variable": "Generator"},
    template="plotly_white",
)
fig.show()

In [ ]:
total_gas_mwh_usage = n.generators_t.p["gas_turbine"].sum()
total_data_centre_mwh_usage = n.loads_t.p["data_centre_load"].sum()
print(
    f"Ratio of data centre load covered by gas turbine: {total_gas_mwh_usage / total_data_centre_mwh_usage:.2%}"
)

In [ ]:
n.model.objective.value

## From Gemini (Nuclear PPA prices)
PPA Market Dynamics: The general PPA market in the UK has experienced volatility. While specific nuclear PPA prices for data centres are not public, the strike price for Hinkley Point C's Contract for Difference (CfD) is £92.50 per MWh (in 2012 prices), which serves as a long-term, inflation-indexed reference for new large-scale nuclear, albeit a subsidised rate. Market analysts have noted that premiums in the UK PPA market can add approximately £10/MWh to baseline PPA costs for corporate buyers seeking specific attributes like 24/7 carbon-free power.

In [ ]:
HOURS_PER_YEAR = 8760  # Total hours in a year
CARBON_FREE_NUCLEAR_PPA_PREMIUM = (
    10  # £/MWh, assumed premium for carbon-free nuclear PPA
)
NUCLEAR_PPA_COST_PER_MWH = 133  # 92.5 £2012/MWh in todays money

for load in DC_LOADS:
    n = networks[load]
    print(f"Data Centre Load: {load} MW")
    cost_of_running_microgrid_per_year = n.model.objective.value
    cost_of_running_with_nuclear_ppa_per_year = (
        load
        * HOURS_PER_YEAR
        * (NUCLEAR_PPA_COST_PER_MWH + CARBON_FREE_NUCLEAR_PPA_PREMIUM)
    )  # £ per year, assuming a PPA of £60/MWh

    print(
        f"Cost of running the microgrid per year: £{cost_of_running_microgrid_per_year:,.2f}"
    )
    print(
        f"Cost of running with a nuclear PPA per year: £{cost_of_running_with_nuclear_ppa_per_year:,.2f}"
    )
    # Calculate the percentage difference in costs
    cost_difference_percentage = (
        (cost_of_running_microgrid_per_year - cost_of_running_with_nuclear_ppa_per_year)
        / cost_of_running_with_nuclear_ppa_per_year
    ) * 100
    print(f"Percentage difference in costs: {cost_difference_percentage:.2f}%\n")

# Plan for the future
1. Initialise data centre load as a constant
2. Loop through wind farms with capacities at different multiple factors of the data centre.
3. Assume constant things like capex cost and capacity factor for wind. We can do a sensitivity analysis of these later.
4. v1: Assume no battery? v2: Assume battery as a MWh of the multiple of the data centre load too.
5. Solve the model to get the least cost way of operating it with gas + battery in each setup. Calculate the % of the total data centre load that was met by gas. Crude calcualtion = MWh of gas / MWh of data centre throughout the year?


## Some options for analysis
* GIVEN a constant data centre load and GIVEN some historical wind profile. Q: How big should the battery and wind farm be priortional to the data centre load to cost-optimally run the system? How much does it cost to run that system? How would that compare to covering the data centre load via a nuclear PPA?
* Alternative route: Given different configurations of wind sizes, but allowing gas to be as much as is needed to cover the data centre load, how much does it cost to run each of those systems? Compared to nuclear PPA?